<a href="https://colab.research.google.com/github/nelsonkam/trading_python/blob/master/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout

In [0]:
ticker = 'TSLA'
df = data.DataReader(ticker, 'iex', datetime(2018, 8, 1), datetime(2018, 9, 1))
df_data = data.DataReader(ticker, 'iex', datetime(2014, 1, 1), datetime(2018, 12, 31))
df_test = data.DataReader(ticker, 'iex', datetime(2019, 1, 1), datetime(2019, 1, 31))

In [0]:
df.head()

In [0]:
df['mid_price'] = (df['low'] + df['high']) / 2
df.plot(y='mid_price')

In [0]:
df_data['mid_price'] = (df_data['low'] + df_data['high']) / 2
data = df_data['mid_price'].values
data

In [0]:
data.shape # (row h, column v)

In [0]:
data = data.reshape(-1, 1) # change array/matrix shape -- same as (x, 1)
data.shape

In [0]:
# Normalize train data
scaler = MinMaxScaler()
scaler.fit(data) # calculates min and max of dataset
scaler.data_max_

In [0]:
data = scaler.transform(data)
data

In [0]:
train_data = data[:]
train_data.size

In [0]:
window_size = 100 # size of batch/window for average calculation
std_avg_predictions = []
std_avg_x = []
mse_errors = []

for index in range(window_size, train_data.size):
    date = df_data.index[index]
        
    prediction = np.mean(train_data[index-window_size:index])
    std_avg_predictions.append(prediction)
    
    mse_errors.append((std_avg_predictions[-1]-train_data[index])**2)
    std_avg_x.append(date)

print('Mean Squared Error for standard averaging: %.5f'%(0.5*np.mean(mse_errors)))

In [0]:
df_result = pd.DataFrame(index=std_avg_x, columns=['prediction', 'true'])
df_result['prediction'] = std_avg_predictions

# the first window_size nth are ignored as they are 
# used by the standard averaging function
df_result['true'] = train_data[window_size:]  
df_result.plot(y=['prediction', 'true'])

In [0]:
# Here we are trying to predict price
# of stock based on the past 60 days.
# So our input (feature_set) here is 
# the prices of the last 60 days and
# the output (label) is the price on
# on the 61st day.
window = 60
features_set = []  
labels = []  
for i in range(window, data.size):  
    features_set.append(data[i-window:i, 0])
    labels.append(data[i, 0])
features_set, labels = np.array(features_set), np.array(labels)  
features_set, labels

In [0]:
# We need to change the shape of
# features_set (1167, 60) to a
# 3D matrix for the LSTM training
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
features_set.shape

In [0]:
model = Sequential()  
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50))  
model.add(Dropout(0.2))

model.add(Dense(units = 1))

In [0]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error') 

In [0]:
model.fit(features_set, labels, epochs = 100, batch_size = 32)

Epoch 1/100
1167/1167 [==============================] - 10s 8ms/step - loss: 0.0592
Epoch 2/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0121
Epoch 3/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0109
Epoch 4/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0101
Epoch 5/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0101
Epoch 6/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0107
Epoch 7/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0084
Epoch 8/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0090
Epoch 9/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0090
Epoch 10/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0077
Epoch 11/100
1167/1167 [==============================] - 6s 5ms/step - loss: 0.0068
Epoch 12/100
1167/1167 [==============================] - 6s 5ms/step - l

In [0]:
df_test['mid_price'] = (df_test['low'] + df_test['high']) / 2
total = pd.concat((df_data['mid_price'], df_test['mid_price']), axis=0) 
total

In [0]:
# Take out prices of the month of January and
# the 60 days before that
test_inputs = total[len(total) - len(df_test) - window:].values
test_inputs

In [0]:
test_inputs = test_inputs.reshape(-1,1)  
test_inputs = scaler.transform(test_inputs)
test_inputs

In [0]:
test_features = []  
for i in range(window, test_inputs.size):  
    test_features.append(test_inputs[i-window:i, 0])
test_features

In [0]:
test_features = np.array(test_features)  
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
test_features.shape

In [0]:
predictions = model.predict(test_features)
predictions = scaler.inverse_transform(predictions)

In [0]:
plt.figure(figsize=(20,6))  
plt.plot(df_test['mid_price'], color='blue', label='Actual Stock Price')  
plt.plot(predictions , color='red', label='Predicted Stock Price')  
plt.title('Stock Price Prediction')  
plt.xlabel('Date')  
plt.ylabel('Stock Price')  
plt.legend()  
plt.show()  